In [153]:
import warnings
warnings.filterwarnings('ignore')

import io
import random
import numpy as np
import mxnet as mx
import gluonnlp as nlp
from bert import data, model
import pandas as pd
import scipy

In [143]:
import bert

In [25]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [49]:
import spacy
spacy.__version__

'2.2.3'

In [144]:
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)
# change `ctx` to `mx.cpu()` if no GPU is available.
#ctx = mx.gpu(0)
ctx = mx.cpu()

In [16]:
pip install sentence-transformers

     |████████████████████████████████| 51kB 851kB/s eta 0:00:011
     |████████████████████████████████| 450kB 1.7MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 2.8MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.5-cp37-none-any.whl size=64943 sha256=9fd8d6c3cc2ceab846e3e826dacca76f293b1d323d3a71116f2ba354bce90982
  Stored in directory: /Users/rahul.b.sarkar/Library/Caches/pip/wheels/b4/ce/39/5bbda8ac34eb52df8c6531382ca077773fbfcbfb6386e5d66c
Successfully built sentence-transformers
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install mxnet-cu100
!pip install bert-embedding --no-deps
!pip install gluonnlp

In [2]:
#pip install mxnet-cu100

In [12]:
pip install tensorflow_hub

     |████████████████████████████████| 92kB 650kB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [74]:
pip install tensorflow==2.0.0-alpha

  Using cached https://files.pythonhosted.org/packages/85/fb/6de0f86efab74e2f29fcc7372f12a59925276842f3c6ff6e032bd7a2bca9/tensorflow-2.0.0a0-cp37-cp37m-macosx_10_11_x86_64.whl
  Found existing installation: tensorflow 1.13.2
    Uninstalling tensorflow-1.13.2:
      Successfully uninstalled tensorflow-1.13.2
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install bert-embedding --no-deps

Note: you may need to restart the kernel to use updated packages.


In [145]:
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12',
                                             dataset_name='book_corpus_wiki_en_uncased',
                                             pretrained=True, ctx=ctx, use_pooler=True,
                                             use_decoder=False, use_classifier=False)
print(bert_base)

BERTModel(
  (encoder): BERTEncoder(
    (dropout_layer): Dropout(p = 0.1, axes=())
    (layer_norm): BERTLayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
    (transformer_cells): HybridSequential(
      (0): BERTEncoderCell(
        (dropout_layer): Dropout(p = 0.1, axes=())
        (attention_cell): MultiHeadAttentionCell(
          (_base_cell): DotProductAttentionCell(
            (_dropout_layer): Dropout(p = 0.1, axes=())
          )
          (proj_query): Dense(768 -> 768, linear)
          (proj_key): Dense(768 -> 768, linear)
          (proj_value): Dense(768 -> 768, linear)
        )
        (proj): Dense(768 -> 768, linear)
        (ffn): BERTPositionwiseFFN(
          (ffn_1): Dense(768 -> 3072, linear)
          (activation): GELU()
          (ffn_2): Dense(3072 -> 768, linear)
          (dropout_layer): Dropout(p = 0.1, axes=())
          (layer_norm): BERTLayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
        )
        (la

In [146]:
bert_classifier = model.classification.BERTClassifier(bert_base, num_classes=2, dropout=0.1)
# only need to initialize the classifier layer.
bert_classifier.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
bert_classifier.hybridize(static_alloc=True)

# softmax cross entropy loss for classification
loss_function = mx.gluon.loss.SoftmaxCELoss()
loss_function.hybridize(static_alloc=True)

metric = mx.metric.Accuracy()

In [8]:
train_df = pd.read_csv('aqsampledata.csv')

In [9]:
train_df.head()

,id,RefAnswer,StudentAnswer,is_similar,qid1,qid2,Marks,Justification,Complexity
0,0.0,A barcode is a series of light and dark bars o...,A barcode is a rectangular strip comprising of...,1.0,1.0,2.0,1,2,tough
1,1.0,With a code number printed underneath,A barcode is a rectangular strip comprising of...,0.0,3.0,4.0,0,0,tough
2,2.0,"Made up of country of origin code, manufacture...",A barcode is a rectangular strip comprising of...,1.0,5.0,6.0,1,1,tough
3,NaN,NaN,A barcode is a rectangular strip comprising of...,NaN,NaN,NaN,1,1,tough
4,NaN,NaN,A barcode is a rectangular strip comprising of...,NaN,NaN,NaN,0,0,easy


In [43]:
#data_train_raw = train_df.RefAnswer + train_df.StudentAnswer
#print(data_train_raw)

In [154]:
tsv_file = io.open('train.tsv', encoding='utf-8')
for i in range(5):
    print(tsv_file.readline())

id	RefAnswer	StudentAnswer	Marks	Justification	Complexity

0	A barcode is a series of light and dark bars of differing widths 	A barcode is a rectangular strip comprising of light and deep colors with a code number printed used for product identification and selling.	1	2	tough

1	With a code number printed underneath	A barcode is a bar with product code printed. It is attached to every product at retail shops.	0	0	tough

2	Made up of country of origin code, manufacturer code, the product code, a check digit.	It is a machine readable numbers arranged in a sequence comprising of parallel lines of different widths on a package. This is used for electronic scanning of goods at checkout to register price.	1	1	tough

3		Barcode is a unique series of parallel lines printed on a product. It is used to register check out price and stock management.	1	1	tough



In [148]:
num_discard_samples = 1
# Split fields by tabs
field_separator = nlp.data.Splitter('\t')
# Fields to select from the file
field_indices = [0, 1, 2]
data_train_raw = nlp.data.TSVDataset(filename='train.tsv',
                                 field_separator=field_separator,
                                 num_discard_samples=num_discard_samples,
                                 field_indices=field_indices)
sample_id = 0

In [149]:
print(data_train_raw)

In [150]:
# Sentence A
print(data_train_raw[sample_id][0])
# Sentence B
print(data_train_raw[sample_id][1])
# 1 means equivalent, 0 means not equivalent
print(data_train_raw[sample_id][2])

0
A barcode is a series of light and dark bars of differing widths 
A barcode is a rectangular strip comprising of light and deep colors with a code number printed used for product identification and selling.


In [80]:

# Use the vocabulary from pre-trained model for tokenization
bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)

# The maximum length of an input sequence
max_len = 128

# The labels for the two classes [(0 = not similar) or  (1 = medium marks) or 2 = correct marks]
all_labels = ["0"]

# whether to transform the data as sentence pairs.
# for single sentence classification, set pair=False
# for regression task, set class_labels=None
# for inference without label available, set has_label=False
pair = True
transform = data.transform.BERTDatasetTransform(bert_tokenizer, max_len,
                                                class_labels=all_labels,
                                                has_label=True,
                                                pad=True,
                                                pair=pair)
data_train = data_train_raw.transform(transform)

print('vocabulary used for tokenization = \n%s'%vocabulary)
print('%s token id = %s'%(vocabulary.padding_token, vocabulary[vocabulary.padding_token]))
print('%s token id = %s'%(vocabulary.cls_token, vocabulary[vocabulary.cls_token]))
print('%s token id = %s'%(vocabulary.sep_token, vocabulary[vocabulary.sep_token]))
print('token ids = \n%s'%data_train[sample_id][0])
print('valid length = \n%s'%data_train[sample_id][1])
print('segment ids = \n%s'%data_train[sample_id][2])
print('label = \n%s'%data_train[sample_id][3])


vocabulary used for tokenization = 
Vocab(size=30522, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
[PAD] token id = 1
[CLS] token id = 2
[SEP] token id = 3


KeyError: 'A barcode is a rectangular strip comprising of light and deep colors with a code number printed used for product identification and selling.'

In [77]:

# Use the vocabulary from pre-trained model for tokenization
bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)

# The maximum length of an input sequence
max_len = 128

# The labels for the two classes [(0 = not similar) or  (1 = medium marks) or 2 = correct marks]
all_labels = ["0", "1", "2", "3"]

# whether to transform the data as sentence pairs.
# for single sentence classification, set pair=False
# for regression task, set class_labels=None
# for inference without label available, set has_label=False
pair = True
transform = data.transform.BERTDatasetTransform(bert_tokenizer, max_len,
                                                class_labels=all_labels,
                                                has_label=True,
                                                pad=True,
                                                pair=pair)
data_train = data_train_raw.transform(transform)

print('vocabulary used for tokenization = \n%s'%vocabulary)
print('%s token id = %s'%(vocabulary.padding_token, vocabulary[vocabulary.padding_token]))
print('%s token id = %s'%(vocabulary.cls_token, vocabulary[vocabulary.cls_token]))
print('%s token id = %s'%(vocabulary.sep_token, vocabulary[vocabulary.sep_token]))
print('token ids = \n%s'%data_train[sample_id][0])
print('valid length = \n%s'%data_train[sample_id][1])
print('segment ids = \n%s'%data_train[sample_id][2])
print('label = \n%s'%data_train[sample_id][3])


vocabulary used for tokenization = 
Vocab(size=30522, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
[PAD] token id = 1
[CLS] token id = 2
[SEP] token id = 3


KeyError: 'A barcode is a rectangular strip comprising of light and deep colors with a code number printed used for product identification and selling.'

In [10]:
# The hyperparameters
batch_size = 32
lr = 5e-6

# The FixedBucketSampler and the DataLoader for making the mini-batches
train_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[1]) for item in data_train],
                                            batch_size=batch_size,
                                            shuffle=True)
bert_dataloader = mx.gluon.data.DataLoader(data_train, batch_sampler=train_sampler)

trainer = mx.gluon.Trainer(bert_classifier.collect_params(), 'adam',
                           {'learning_rate': lr, 'epsilon': 1e-9})

# Collect all differentiable parameters
# `grad_req == 'null'` indicates no gradients are calculated (e.g. constant parameters)
# The gradients for these params are clipped later
params = [p for p in bert_classifier.collect_params().values() if p.grad_req != 'null']
grad_clip = 1

# Training the model with only three epochs
log_interval = 4
num_epochs = 3
for epoch_id in range(num_epochs):
    metric.reset()
    step_loss = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(bert_dataloader):
        with mx.autograd.record():

            # Load the data to the GPU
            token_ids = token_ids.as_in_context(ctx)
            valid_length = valid_length.as_in_context(ctx)
            segment_ids = segment_ids.as_in_context(ctx)
            label = label.as_in_context(ctx)

            # Forward computation
            out = bert_classifier(token_ids, segment_ids, valid_length.astype('float32'))
            ls = loss_function(out, label).mean()

        # And backwards computation
        ls.backward()

        # Gradient clipping
        trainer.allreduce_grads()
        nlp.utils.clip_grad_global_norm(params, 1)
        trainer.update(1)

        step_loss += ls.asscalar()
        metric.update([label], [out])

        # Printing vital information
        if (batch_id + 1) % (log_interval) == 0:
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.7f}, acc={:.3f}'
                         .format(epoch_id, batch_id + 1, len(bert_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, metric.get()[1]))
            step_loss = 0


[Epoch 0 Batch 4/6] loss=0.6419, lr=0.0000050, acc=0.492
[Epoch 1 Batch 4/6] loss=0.7024, lr=0.0000050, acc=0.375
[Epoch 2 Batch 4/6] loss=0.7097, lr=0.0000050, acc=0.594


In [128]:
from sentence_transformers import SentenceTransformer

# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md

model = SentenceTransformer('bert-base-nli-mean-tokens')


In [129]:
#sentence_df = pd.read_csv('train_bert.csv',engine='python')
sentence_df = pd.read_csv('train_universal_sentence.csv',engine='python')

In [130]:
sentence_list = sentence_df.RefAnswer.astype(str).tolist()

In [131]:
print(sentence_list)

['A barcode is a series of light and dark bars of differing widths ', 'With a code number printed underneath', 'Made up of country of origin code, manufacturer code, the product code, a check digit.', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan']


In [132]:
# sentences = ['A barcode is a series of light and dark bars of differing widths', 
#              'With a code number printed underneath',
#              'Made up of country of origin code, manufacturer code, the product code, a check digit.']

# Each sentence is encoded as a 1-D vector with 78 columns
sentence_embeddings = model.encode(sentence_list)

print('Sample BERT embedding vector - length', len(sentence_embeddings[0]))

print('Sample BERT embedding vector - note includes negative values', sentence_embeddings[0])

Sample BERT embedding vector - length 768
Sample BERT embedding vector - note includes negative values [ 9.15223420e-01  4.33433205e-01  3.87063809e-03  3.62613618e-01
  5.39999843e-01  5.09764671e-01  5.00931263e-01  2.77469307e-02
 -3.76719952e-01 -6.67793572e-01 -7.19707906e-01 -2.09060386e-01
 -2.01652005e-01  6.28048956e-01  7.39025474e-01  7.78510869e-01
  1.95034966e-01 -4.08038944e-02  2.60271579e-02 -2.20886350e-01
 -8.19787160e-02  8.56281281e-01 -6.85880005e-01 -1.09536421e+00
  2.08608598e-01  2.14172397e-02  2.01437071e-01  5.61132729e-02
 -7.94450462e-01 -2.12467849e-01  7.94746757e-01  7.40739763e-01
 -6.82536466e-03  6.25642715e-03  7.45293617e-01 -6.95767403e-02
  7.14476183e-02 -4.05434728e-01  1.03005624e+00  2.79981554e-01
  1.10992229e+00 -5.36680877e-01  6.19239569e-01 -5.54652512e-01
 -1.88014597e-01 -6.52552247e-01  3.76414061e-01 -4.20311511e-01
 -8.40249002e-01 -1.80035219e-01 -3.29662055e-01  1.65305451e-01
  6.87114000e-01  1.05758023e+00 -3.56784016e-01  4.

In [32]:
#sentences

In [68]:
#query_embeddings = model.encode(queries)

In [34]:
#print(query_embeddings)

In [97]:
#queries = sentence_df.StudentAnswer.astype(str).tolist()
#query_embeddings = model.encode(queries)

In [133]:
test_df = pd.read_csv('test_universal_sentence.csv',engine='python')

In [134]:
queries = test_df.StudentAnswer

In [35]:
#queries = ['Barcode is a unique series of parallel lines printed on a product. It is used to register check out price and stock management.', 'A barcode is a way to encode information visually that a machine can read. The combination of black and white bars (elements) represents a code for product identification.', 'A barcode is a series of light and dark bars of differing thickness.']



In [135]:
query_embeddings = model.encode(queries)

In [137]:
# Find the closest 3 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 10
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 3 most similar sentences in corpus:")
    
    results_list = []

    for idx, distance in results[0:closest_n]:
        score = sentence_list[idx].strip(), "(Score: %.4f)" % (1-distance)
        print(score)
        score_value = float((1-distance))
        print(score_value)
        if score_value[] >= 0.6:
            results_list.append(score_value)
        print(results_list)
        #if float(score) > 0.6:
            #results_list.add(0.6)
            #print(results_list)
            
        





Query: A barcode consists of light and dark bars of different widths, with a code number printed below the bars.

Top 3 most similar sentences in corpus:
('A barcode is a series of light and dark bars of differing widths', '(Score: 0.9105)')
0.9105138121526059
[0.9105138121526059]
('With a code number printed underneath', '(Score: 0.6889)')
0.6888538652669248
[0.9105138121526059, 0.6888538652669248]
('Made up of country of origin code, manufacturer code, the product code, a check digit.', '(Score: 0.5022)')
0.5022032782791848
[0.9105138121526059, 0.6888538652669248]
('nan', '(Score: 0.1636)')
0.1635611849015829
[0.9105138121526059, 0.6888538652669248]
('nan', '(Score: 0.1636)')
0.1635611849015829
[0.9105138121526059, 0.6888538652669248]
('nan', '(Score: 0.1636)')
0.1635611849015829
[0.9105138121526059, 0.6888538652669248]
('nan', '(Score: 0.1636)')
0.1635611849015829
[0.9105138121526059, 0.6888538652669248]
('nan', '(Score: 0.1636)')
0.16356116075015392
[0.9105138121526059, 0.68885

In [139]:
#print(results)

# if results.astype(float) >= 0.6:
#     print("OK")
# else:
#     print("Negative")
    
print(results_list) 
 

[]


In [117]:
#for results in 